<a href="https://colab.research.google.com/github/2303a52239/HPC/blob/main/hpc_lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   **Roll no:2303A52339**
2.   **Batch:35**


1.   **name:vyshnavi siliveru**







In [1]:
import threading

N = 100
T = 4

def worker(tid):
    start = tid * (N // T)
    end = (tid + 1) * (N // T)
    for i in range(start, end):
        print(f"Thread {tid} → Iteration {i}")

threads = []
for t in range(T):
    th = threading.Thread(target=worker, args=(t,))
    threads.append(th)
    th.start()

for th in threads:
    th.join()


Thread 0 → Iteration 0
Thread 0 → Iteration 1
Thread 0 → Iteration 2
Thread 0 → Iteration 3
Thread 0 → Iteration 4
Thread 0 → Iteration 5
Thread 0 → Iteration 6
Thread 0 → Iteration 7
Thread 0 → Iteration 8
Thread 0 → Iteration 9
Thread 0 → Iteration 10
Thread 0 → Iteration 11
Thread 0 → Iteration 12
Thread 0 → Iteration 13
Thread 0 → Iteration 14
Thread 0 → Iteration 15
Thread 0 → Iteration 16
Thread 0 → Iteration 17
Thread 0 → Iteration 18
Thread 0 → Iteration 19
Thread 0 → Iteration 20
Thread 0 → Iteration 21
Thread 0 → Iteration 22
Thread 0 → Iteration 23
Thread 0 → Iteration 24
Thread 1 → Iteration 25
Thread 1 → Iteration 26
Thread 1 → Iteration 27
Thread 1 → Iteration 28
Thread 1 → Iteration 29
Thread 1 → Iteration 30
Thread 1 → Iteration 31
Thread 1 → Iteration 32
Thread 1 → Iteration 33
Thread 1 → Iteration 34
Thread 1 → Iteration 35
Thread 1 → Iteration 36
Thread 1 → Iteration 37
Thread 1 → Iteration 38
Thread 1 → Iteration 39
Thread 1 → Iteration 40
Thread 1 → Iteration 41
Th

In [2]:
import numpy as np
from numba import njit, prange

@njit(parallel=True)
def static_sum(A):
    s = 0.0
    for i in prange(len(A)):
        s += A[i]
    return s

A = np.random.rand(10_000_000)
print(static_sum(A))


4999424.498390624


In [3]:
import numpy as np
import time
from numba import njit, prange

N = 20_000_000
A = np.random.rand(N)
B = np.random.rand(N)
C = np.zeros(N)

# ----------------------------
# Python Serial
# ----------------------------
def python_serial(A, B, C):
    for i in range(len(A)):
        C[i] = A[i] + B[i]

# ----------------------------
# Numba Serial
# ----------------------------
@njit
def numba_serial(A, B, C):
    for i in range(len(A)):
        C[i] = A[i] + B[i]

# ----------------------------
# Numba Parallel (Static)
# ----------------------------
@njit(parallel=True)
def numba_parallel(A, B, C):
    for i in prange(len(A)):
        C[i] = A[i] + B[i]

# Warm-up (important!)
numba_serial(A, B, C)
numba_parallel(A, B, C)

# ----------------------------
# Timing
# ----------------------------
t1 = time.time()
python_serial(A, B, C)
t2 = time.time()

t3 = time.time()
numba_serial(A, B, C)
t4 = time.time()

t5 = time.time()
numba_parallel(A, B, C)
t6 = time.time()

print("Python Serial Time  :", t2 - t1)
print("Numba Serial Time   :", t4 - t3)
print("Numba Parallel Time :", t6 - t5)


Python Serial Time  : 8.238338470458984
Numba Serial Time   : 0.0389101505279541
Numba Parallel Time : 0.03609275817871094


In [4]:
import numpy as np
import time
import threading
from numba import njit, prange

# Matrix size (keep moderate for Python)
N = 300

A = np.random.rand(N, N)
B = np.random.rand(N, N)

C_py_serial = np.zeros((N, N))
C_py_parallel = np.zeros((N, N))
C_numba_serial = np.zeros((N, N))
C_numba_parallel = np.zeros((N, N))

# --------------------------------
# Python Serial
# --------------------------------
def python_serial_matmul(A, B, C):
    for i in range(N):
        for j in range(N):
            s = 0.0
            for k in range(N):
                s += A[i, k] * B[k, j]
            C[i, j] = s

# --------------------------------
# Python Parallel (Row-wise)
# --------------------------------
def worker(start, end):
    for i in range(start, end):
        for j in range(N):
            s = 0.0
            for k in range(N):
                s += A[i, k] * B[k, j]
            C_py_parallel[i, j] = s

def python_parallel_matmul(A, B):
    threads = []
    T = 4
    rows_per_thread = N // T

    for t in range(T):
        start = t * rows_per_thread
        end = (t + 1) * rows_per_thread
        th = threading.Thread(target=worker, args=(start, end))
        threads.append(th)
        th.start()

    for th in threads:
        th.join()

# --------------------------------
# Numba Serial
# --------------------------------
@njit
def numba_serial_matmul(A, B, C):
    for i in range(N):
        for j in range(N):
            s = 0.0
            for k in range(N):
                s += A[i, k] * B[k, j]
            C[i, j] = s

# --------------------------------
# Numba Parallel (Static Scheduling)
# --------------------------------
@njit(parallel=True)
def numba_parallel_matmul(A, B, C):
    for i in prange(N):   # static scheduling
        for j in range(N):
            s = 0.0
            for k in range(N):
                s += A[i, k] * B[k, j]
            C[i, j] = s

# --------------------------------
# Warm-up (Numba compilation)
# --------------------------------
numba_serial_matmul(A, B, C_numba_serial)
numba_parallel_matmul(A, B, C_numba_parallel)

# --------------------------------
# Timing
# --------------------------------

# Python Serial
t1 = time.time()
python_serial_matmul(A, B, C_py_serial)
t2 = time.time()

# Python Parallel
t3 = time.time()
python_parallel_matmul(A, B)
t4 = time.time()

# Numba Serial
t5 = time.time()
numba_serial_matmul(A, B, C_numba_serial)
t6 = time.time()

# Numba Parallel
t7 = time.time()
numba_parallel_matmul(A, B, C_numba_parallel)
t8 = time.time()

# --------------------------------
# Output
# --------------------------------
print("Python Serial Time        :", t2 - t1, "seconds")
print("Python Parallel Time      :", t4 - t3, "seconds")
print("Numba Serial Time         :", t6 - t5, "seconds")
print("Numba Parallel Time       :", t8 - t7, "seconds")


Python Serial Time        : 13.005486965179443 seconds
Python Parallel Time      : 12.84059190750122 seconds
Numba Serial Time         : 0.04278922080993652 seconds
Numba Parallel Time       : 0.02373504638671875 seconds
